# Loading data with TimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import TimeBasedConfig # Time based dataset MUST use TimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
time_based_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.TIME_BASED, display_details=True)

[2025-08-19 12:08:28,172][wrapper_dataset][INFO] - Dataset is time-based. Use cesnet_tszoo.configs.TimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(ts_ids, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(ts_ids, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:28,177][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:28,197][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:28,201][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 470.79it/s]
[2025-08-19 12:08:28,323][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [231 123 172 368 290 ... 156 504 169 274  26], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
time_based_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")
# Or
time_based_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")

[2025-08-19 12:08:28,331][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 12:08:28,332][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 12:08:28,333][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 12:08:28,334][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 12:08:28,334][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
time_based_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
# Or
time_based_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)

[2025-08-19 12:08:28,350][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 12:08:28,350][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 12:08:28,351][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 12:08:28,353][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 12:08:28,353][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 12:08:28,364][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 206.04it/s]


(54, 32, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = time_based_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 12:08:28,886][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 32/32 [00:00<00:00, 136.91it/s]


(54, 64, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = time_based_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 12:08:29,134][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 102.10it/s]


(54, 128, 20)

#### All set

- Affected by `all_batch_size`.
- Affected by `all_workers`.

In [10]:
dataloader = time_based_dataset.get_all_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 12:08:29,252][cesnet_dataset][INFO] - Created new cached all_dataloader.
100%|██████████| 53/53 [00:00<00:00, 178.20it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [11]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:29,558][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:29,581][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:29,585][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 794.73it/s]
[2025-08-19 12:08:29,656][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [ 66 203 212 475 281 ...   0 476 262 157 359], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
   

In [12]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-08-19 12:08:29,664][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 237.86it/s]


(54, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [13]:
config = TimeBasedConfig(ts_ids=[177, 176, 319, 267], train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:30,116][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:30,135][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:30,139][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 4/4 [00:00<00:00, 614.84it/s]
[2025-08-19 12:08:30,147][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [177 176 319 267], Length=4
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Slidin

In [14]:
dataloader = time_based_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-08-19 12:08:30,154][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 1033.39it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(ts_ids, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(ts_ids, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [15]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:30,265][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:30,283][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:30,286][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 2202.74it/s]
[2025-08-19 12:08:30,314][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [171 282  99 274 359 ...  77 443 126 305 542], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 335
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
        Train batch size: 32
        Val batch size: 64
        Test batch size: 128
        All batch size: 128
    D

In [16]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-08-19 12:08:30,322][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 5089.34it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [17]:
time_based_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
time_based_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-08-19 12:08:30,424][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 12:08:30,424][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-08-19 12:08:30,425][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 12:08:30,425][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-08-19 12:08:30,426][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-08-19 12:08:30,426][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [18]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:30,432][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:30,451][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:30,454][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 1961.09it/s]
[2025-08-19 12:08:30,485][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [369  62 232 161 423 ... 538 295 167 386 233], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 100
    Fillers
        Filler type: None
    Transformers
        Transformer type: None
    Batch sizes
        Train batch size: 32
        Val batch size: 64
        Test batch size: 128
        All batch size: 128
    

In [19]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-08-19 12:08:30,493][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 4677.85it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [20]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:30,605][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:30,624][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:30,628][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 674.15it/s]
[2025-08-19 12:08:30,711][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [206 539 107 521 492 ...  59 256 356 433 430], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

#### Train set

- Affected by `train_workers`.

In [21]:
df = time_based_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,206.0,0.0,23597.0,428294.0,2.381077e+08,6579.0,12.98,26.500000,12504.0,24.66,56.689999,14342.0,28.290001,53.939999,0.660156,0.709961,0.449951,0.469971,12.97,140.039993
1,206.0,1.0,25923.0,1210079.0,1.065633e+09,6933.0,13.87,28.170000,13295.0,26.59,59.320000,15404.0,30.809999,58.709999,0.669922,0.720215,0.469971,0.489990,13.51,135.059998
2,206.0,2.0,24718.0,989729.0,8.968168e+08,6606.0,13.13,26.709999,12035.0,23.93,52.639999,14615.0,29.059999,58.990002,0.669922,0.709961,0.469971,0.489990,12.67,136.250000
3,206.0,3.0,23330.0,736273.0,6.491255e+08,6124.0,12.13,25.270000,11020.0,21.82,49.529999,13742.0,27.209999,61.820000,0.689941,0.729980,0.479980,0.509766,12.96,133.759995
4,206.0,4.0,21234.0,983502.0,6.705902e+08,5622.0,11.24,22.670000,8829.0,17.66,39.389999,12933.0,25.870001,63.270000,0.669922,0.700195,0.469971,0.479980,13.50,129.929993
5,206.0,5.0,19504.0,1624940.0,1.490053e+09,5296.0,10.47,21.879999,7148.0,14.13,31.809999,12178.0,24.070000,65.730003,0.649902,0.680176,0.469971,0.479980,14.27,131.240005
6,206.0,6.0,18605.0,683014.0,4.911213e+08,5330.0,10.58,22.170000,7155.0,14.20,31.440001,11965.0,23.740000,64.019997,0.660156,0.680176,0.469971,0.479980,14.70,131.970001
7,206.0,7.0,19204.0,541664.0,3.901084e+08,5414.0,10.55,22.330000,8074.0,15.74,35.310001,12068.0,23.520000,61.560001,0.669922,0.700195,0.449951,0.469971,13.53,133.539993
8,206.0,8.0,19237.0,1612057.0,1.421304e+09,5393.0,10.49,22.879999,8199.0,15.95,36.790001,12048.0,23.440001,60.650002,0.669922,0.700195,0.459961,0.469971,14.20,129.309998
9,206.0,9.0,20286.0,2307641.0,1.853283e+09,5418.0,10.64,22.580000,8322.0,16.35,36.549999,12339.0,24.240000,60.889999,0.660156,0.689941,0.479980,0.479980,14.59,128.529999


In [22]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets       n_bytes  \
 0                     206.0      0.0  23597.0   428294.0  2.381077e+08   
 1                     206.0      1.0  25923.0  1210079.0  1.065633e+09   
 2                     206.0      2.0  24718.0   989729.0  8.968168e+08   
 3                     206.0      3.0  23330.0   736273.0  6.491255e+08   
 4                     206.0      4.0  21234.0   983502.0  6.705902e+08   
 ...                     ...      ...      ...        ...           ...   
 3354                  206.0   3354.0  27971.0   341501.0  1.772465e+08   
 3355                  206.0   3355.0  33608.0   600669.0  3.962526e+08   
 3356                  206.0   3356.0  30484.0  2805892.0  3.125735e+09   
 3357                  206.0   3357.0  38880.0   482625.0  1.476098e+08   
 3358                  206.0   3358.0  33151.0   271793.0  7.177263e+07   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             6579.0  

#### Val set

- Affected by `val_workers`.

In [23]:
df = time_based_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,206.0,3359.0,34830.0,291116.0,9.016516e+07,6654.0,13.44,28.540001,16164.0,32.650002,66.580002,14905.0,30.110001,55.320000,0.640137,0.689941,0.479980,0.500000,14.36,119.529999
1,206.0,3360.0,44669.0,406890.0,1.254212e+08,6855.0,13.82,29.730000,23846.0,48.080002,204.339996,15581.0,31.410000,58.230000,0.620117,0.669922,0.489990,0.529785,14.31,123.110001
2,206.0,3361.0,40542.0,360128.0,1.324320e+08,6872.0,13.91,30.049999,19874.0,40.230000,81.730003,15532.0,31.440001,59.000000,0.629883,0.669922,0.479980,0.509766,14.58,122.449997
3,206.0,3362.0,39197.0,308593.0,9.788400e+07,6743.0,13.68,29.350000,18603.0,37.730000,73.050003,15319.0,31.070000,59.049999,0.629883,0.680176,0.479980,0.509766,13.73,122.669998
4,206.0,3363.0,40797.0,378683.0,1.623438e+08,6874.0,13.94,29.830000,19587.0,39.730000,101.040001,15506.0,31.450001,61.680000,0.649902,0.700195,0.479980,0.509766,13.10,123.709999
5,206.0,3364.0,35195.0,918597.0,8.558387e+08,6063.0,12.37,26.459999,17934.0,36.599998,109.919998,13056.0,26.639999,55.900002,0.629883,0.680176,0.479980,0.500000,14.41,125.769997
6,206.0,3365.0,23729.0,486616.0,3.448511e+08,5609.0,11.42,24.680000,12354.0,25.160000,63.380001,12340.0,25.129999,57.000000,0.640137,0.680176,0.469971,0.489990,14.32,125.589996
7,206.0,3366.0,23610.0,1161940.0,6.984892e+08,5548.0,11.30,24.570000,10380.0,21.139999,44.049999,12861.0,26.190001,67.339996,0.620117,0.660156,0.479980,0.489990,14.77,122.010002
8,206.0,3367.0,26484.0,1291071.0,1.000916e+09,5900.0,11.97,26.530001,9853.0,19.990000,42.990002,14002.0,28.400000,74.070000,0.629883,0.669922,0.489990,0.500000,16.40,120.839996
9,206.0,3368.0,32013.0,1450869.0,1.300905e+09,5919.0,11.86,26.000000,11657.0,23.360001,49.279999,13976.0,28.010000,70.919998,0.629883,0.660156,0.479980,0.500000,17.85,118.709999


In [24]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     206.0   3359.0  34830.0   291116.0   90165164.0   
 1                     206.0   3360.0  44669.0   406890.0  125421175.0   
 2                     206.0   3361.0  40542.0   360128.0  132431985.0   
 3                     206.0   3362.0  39197.0   308593.0   97884002.0   
 4                     206.0   3363.0  40797.0   378683.0  162343831.0   
 ...                     ...      ...      ...        ...          ...   
 2010                  206.0   5369.0  29107.0   473971.0  242892556.0   
 2011                  206.0   5370.0  29965.0   344302.0  110901726.0   
 2012                  206.0   5371.0  28087.0   368047.0  194872253.0   
 2013                  206.0   5372.0  32112.0   385510.0  152172968.0   
 2014                  206.0   5373.0  38677.0   416711.0  153915351.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             6654.0           13.

#### Test set

- Affected by `test_workers`.

In [25]:
df = time_based_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,206.0,5374.0,28071.0,295486.0,9.512346e+07,6411.0,12.72,28.469999,14927.0,29.620001,66.919998,15479.0,30.709999,58.080002,0.580078,0.609863,0.439941,0.459961,15.62,124.540001
1,206.0,5375.0,29994.0,343764.0,1.679329e+08,6254.0,12.26,27.760000,16184.0,31.730000,72.660004,15305.0,30.010000,60.540001,0.569824,0.609863,0.439941,0.449951,14.97,128.830002
2,206.0,5376.0,27952.0,276611.0,1.114610e+08,6168.0,12.29,28.240000,14391.0,28.670000,69.760002,14949.0,29.780001,59.290001,0.560059,0.589844,0.439941,0.449951,15.39,125.040001
3,206.0,5377.0,26819.0,250700.0,8.883612e+07,6238.0,12.43,27.459999,13390.0,26.670000,64.300003,15218.0,30.309999,59.740002,0.560059,0.589844,0.459961,0.469971,15.63,127.029999
4,206.0,5378.0,26573.0,2317355.0,2.925078e+09,6018.0,12.08,26.950001,12119.0,24.340000,59.889999,15075.0,30.270000,63.540001,0.549805,0.589844,0.439941,0.459961,15.91,126.589996
5,206.0,5379.0,25373.0,610343.0,4.239582e+08,5804.0,11.84,25.010000,11784.0,24.049999,56.340000,14329.0,29.240000,61.799999,0.569824,0.589844,0.449951,0.459961,17.67,126.650002
6,206.0,5380.0,29405.0,1025369.0,8.811557e+08,5932.0,11.89,25.160000,10535.0,21.110001,48.430000,14989.0,30.040001,72.320000,0.580078,0.609863,0.449951,0.459961,17.17,125.779999
7,206.0,5381.0,25725.0,2981690.0,3.360514e+09,5951.0,12.19,26.219999,10133.0,20.760000,47.189999,14929.0,30.590000,75.430000,0.569824,0.589844,0.449951,0.459961,17.92,123.239998
8,206.0,5382.0,26809.0,2032794.0,2.127514e+09,5919.0,11.81,25.700001,10088.0,20.139999,45.720001,15389.0,30.719999,78.519997,0.580078,0.620117,0.439941,0.449951,17.02,125.379997
9,206.0,5383.0,28886.0,1203530.0,8.530947e+08,6240.0,12.63,29.290001,11090.0,22.450001,52.619999,16108.0,32.610001,84.070000,0.589844,0.609863,0.429932,0.439941,16.66,126.550003


In [26]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets       n_bytes  \
 0                     206.0   5374.0  28071.0   295486.0  9.512346e+07   
 1                     206.0   5375.0  29994.0   343764.0  1.679329e+08   
 2                     206.0   5376.0  27952.0   276611.0  1.114610e+08   
 3                     206.0   5377.0  26819.0   250700.0  8.883612e+07   
 4                     206.0   5378.0  26573.0  2317355.0  2.925078e+09   
 ...                     ...      ...      ...        ...           ...   
 1338                  206.0   6712.0  23912.0   429377.0  1.511826e+08   
 1339                  206.0   6713.0  22323.0   606877.0  4.578363e+08   
 1340                  206.0   6714.0  23565.0   305976.0  9.797108e+07   
 1341                  206.0   6715.0  23295.0   400156.0  1.964785e+08   
 1342                  206.0   6716.0  21303.0   255091.0  8.189589e+07   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             6411.0  

In [27]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets       n_bytes  \
 0                     206.0   5374.0  28071.0   295486.0  9.512346e+07   
 1                     206.0   5375.0  29994.0   343764.0  1.679329e+08   
 2                     206.0   5376.0  27952.0   276611.0  1.114610e+08   
 3                     206.0   5377.0  26819.0   250700.0  8.883612e+07   
 4                     206.0   5378.0  26573.0  2317355.0  2.925078e+09   
 ...                     ...      ...      ...        ...           ...   
 1338                  206.0   6712.0  23912.0   429377.0  1.511826e+08   
 1339                  206.0   6713.0  22323.0   606877.0  4.578363e+08   
 1340                  206.0   6714.0  23565.0   305976.0  9.797108e+07   
 1341                  206.0   6715.0  23295.0   400156.0  1.964785e+08   
 1342                  206.0   6716.0  21303.0   255091.0  8.189589e+07   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             6411.0  

#### All set

- Affected by `all_workers`.

In [28]:
df = time_based_dataset.get_all_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_all_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,206.0,0.0,23597.0,428294.0,2.381077e+08,6579.0,12.98,26.500000,12504.0,24.66,56.689999,14342.0,28.290001,53.939999,0.660156,0.709961,0.449951,0.469971,12.97,140.039993
1,206.0,1.0,25923.0,1210079.0,1.065633e+09,6933.0,13.87,28.170000,13295.0,26.59,59.320000,15404.0,30.809999,58.709999,0.669922,0.720215,0.469971,0.489990,13.51,135.059998
2,206.0,2.0,24718.0,989729.0,8.968168e+08,6606.0,13.13,26.709999,12035.0,23.93,52.639999,14615.0,29.059999,58.990002,0.669922,0.709961,0.469971,0.489990,12.67,136.250000
3,206.0,3.0,23330.0,736273.0,6.491255e+08,6124.0,12.13,25.270000,11020.0,21.82,49.529999,13742.0,27.209999,61.820000,0.689941,0.729980,0.479980,0.509766,12.96,133.759995
4,206.0,4.0,21234.0,983502.0,6.705902e+08,5622.0,11.24,22.670000,8829.0,17.66,39.389999,12933.0,25.870001,63.270000,0.669922,0.700195,0.469971,0.479980,13.50,129.929993
5,206.0,5.0,19504.0,1624940.0,1.490053e+09,5296.0,10.47,21.879999,7148.0,14.13,31.809999,12178.0,24.070000,65.730003,0.649902,0.680176,0.469971,0.479980,14.27,131.240005
6,206.0,6.0,18605.0,683014.0,4.911213e+08,5330.0,10.58,22.170000,7155.0,14.20,31.440001,11965.0,23.740000,64.019997,0.660156,0.680176,0.469971,0.479980,14.70,131.970001
7,206.0,7.0,19204.0,541664.0,3.901084e+08,5414.0,10.55,22.330000,8074.0,15.74,35.310001,12068.0,23.520000,61.560001,0.669922,0.700195,0.449951,0.469971,13.53,133.539993
8,206.0,8.0,19237.0,1612057.0,1.421304e+09,5393.0,10.49,22.879999,8199.0,15.95,36.790001,12048.0,23.440001,60.650002,0.669922,0.700195,0.459961,0.469971,14.20,129.309998
9,206.0,9.0,20286.0,2307641.0,1.853283e+09,5418.0,10.64,22.580000,8322.0,16.35,36.549999,12339.0,24.240000,60.889999,0.660156,0.689941,0.479980,0.479980,14.59,128.529999


In [29]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets       n_bytes  \
 0                     206.0      0.0  23597.0   428294.0  2.381077e+08   
 1                     206.0      1.0  25923.0  1210079.0  1.065633e+09   
 2                     206.0      2.0  24718.0   989729.0  8.968168e+08   
 3                     206.0      3.0  23330.0   736273.0  6.491255e+08   
 4                     206.0      4.0  21234.0   983502.0  6.705902e+08   
 ...                     ...      ...      ...        ...           ...   
 6712                  206.0   6712.0  23912.0   429377.0  1.511826e+08   
 6713                  206.0   6713.0  22323.0   606877.0  4.578363e+08   
 6714                  206.0   6714.0  23565.0   305976.0  9.797108e+07   
 6715                  206.0   6715.0  23295.0   400156.0  1.964785e+08   
 6716                  206.0   6716.0  21303.0   255091.0  8.189589e+07   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0             6579.0  

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [30]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:32,376][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:32,395][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:32,399][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 669.76it/s]
[2025-08-19 12:08:32,480][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [448 147 380  99 490 ...  77 261 391 471 166], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    

#### Train set

- Affected by `train_workers`.

In [31]:
numpy_array = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(54, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [32]:
numpy_array = time_based_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(54, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [33]:
numpy_array = time_based_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### All set

- Affected by `all_workers`.

In [34]:
numpy_array = time_based_dataset.get_all_numpy(workers="config")

display(numpy_array.shape)

(54, 6717, 20)

#### Using time_format=TimeFormat.DATETIME

In [35]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-08-19 12:08:32,743][time_config][INFO] - Quick validation succeeded.
[2025-08-19 12:08:32,765][time_config][INFO] - Finalization and validation completed successfully.
[2025-08-19 12:08:32,769][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 1240.09it/s]
[2025-08-19 12:08:32,814][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [250 453 263 373 247 ... 321 180 395 279 160], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
   

In [36]:
numpy_array, times = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(54, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)